<a href="https://colab.research.google.com/github/gregor-posadas/Gregor-Projects/blob/main/WWTP_Heatmap_Correlation_Program_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats
from statsmodels.stats.multitest import multipletests
from sklearn.utils import resample
import itertools


# Function to remove outliers using Z-score
def remove_outliers_zscore(df, threshold=3):
    """
    Remove outliers from a DataFrame using the Z-score method.
    """
    print("Applying Z-Score Method to filter outliers...")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    z_scores = np.abs(stats.zscore(df[numeric_cols], nan_policy="omit"))
    mask = (z_scores < threshold).all(axis=1)
    return df[mask]


# Sequential file upload
def upload_files_sequentially():
    process_files = []
    process_labels = []

    while True:
        process_file = input("Enter the full path to the next process file (e.g., Influent): ")
        process_label = input(f"What is the name of this process? (e.g., Influent): ")

        process_files.append(process_file)
        process_labels.append(process_label)

        add_another = input("Do you want to add another process file? (yes/no): ").strip().lower()
        if add_another != 'yes':
            break

    return process_files, process_labels


# Read and align datasets with Z-score filtering
def read_file(file_path):
    """
    Read an Excel or CSV file into a DataFrame and normalize column names, with Z-score filtering.
    """
    try:
        print(f"Reading file: {file_path}")
        if file_path.endswith(".xlsx") or file_path.endswith(".xls"):
            df = pd.read_excel(file_path)
        elif file_path.endswith(".csv"):
            df = pd.read_csv(file_path)
        else:
            raise ValueError("Unsupported file format. Please use .xlsx, .xls, or .csv files.")

        df.columns = df.columns.str.lower().str.strip()
        if 'date' not in df.columns:
            raise KeyError(f"The file {file_path} does not contain a 'date' column.")
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df = df.dropna(subset=["date"])  # Drop rows with invalid dates
        df = remove_outliers_zscore(df)  # Apply Z-score filtering
        print(f"Finished processing file: {file_path}")
        return df
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        raise


def align_datasets(file1, file2, label1, label2):
    """
    Align two datasets based on the 'date' column, with Z-score filtering.
    """
    print(f"Aligning datasets for {label1} and {label2}...")
    df1 = read_file(file1)
    df2 = read_file(file2)

    merged_df = pd.merge(df1, df2, on="date", suffixes=(f"_{label1}", f"_{label2}"))
    merged_df = merged_df.set_index("date")
    print(f"Datasets aligned for {label1} and {label2}.")
    return merged_df


# Statistical analysis
def bootstrap_correlations(df, n_iterations=500, method='pearson'):
    correlations = []
    for i in range(n_iterations):
        if i % 100 == 0:
            print(f"Bootstrapping iteration {i}/{n_iterations}...")
        df_resampled = resample(df)
        corr_matrix = df_resampled.corr(method=method)
        correlations.append(corr_matrix)

    median_corr = pd.concat(correlations).groupby(level=0).median()
    return median_corr


def calculate_p_values(df, method='pearson'):
    print("Calculating p-values for correlation matrix...")
    p_values = pd.DataFrame(np.ones((df.shape[1], df.shape[1])), columns=df.columns, index=df.columns)
    for col1, col2 in itertools.combinations(df.columns, 2):
        _, p_val = stats.pearsonr(df[col1], df[col2])
        p_values.at[col1, col2] = p_val
        p_values.at[col2, col1] = p_val
    return p_values


def correct_p_values(p_values):
    print("Correcting p-values using Benjamini-Hochberg method...")
    _, corrected, _, _ = multipletests(p_values.values.flatten(), alpha=0.05, method='fdr_bh')
    corrected_p = pd.DataFrame(corrected.reshape(p_values.shape), index=p_values.index, columns=p_values.columns)
    return corrected_p


# Generate Heatmap
def generate_interactive_heatmap(df, title, labels):
    """
    Generate an interactive heatmap with consistent axis alignment and diagonal of 1.00.
    """
    # Preprocess data: Ensure all columns are numeric
    df = df.apply(pd.to_numeric, errors='coerce').dropna(axis=1, how='all').dropna()

    print("Bootstrapping correlation matrices...")
    pearson_corr = bootstrap_correlations(df, n_iterations=500, method='pearson')
    spearman_corr = bootstrap_correlations(df, n_iterations=500, method='spearman')
    kendall_corr = bootstrap_correlations(df, n_iterations=500, method='kendall')

    avg_corr_matrix = (pearson_corr + spearman_corr + kendall_corr) / 3

    print("Calculating and correcting p-values...")
    pearson_p_values = calculate_p_values(df, method='pearson')
    corrected_p_values = correct_p_values(pearson_p_values)

    sig_mask = (corrected_p_values < 0.05).astype(int)

    # Filter the correlation matrix
    filtered_corr_matrix = avg_corr_matrix.where(sig_mask > 0).fillna(0)

    # Ensure row and column order matches
    parameter_order = sorted(filtered_corr_matrix.index)
    filtered_corr_matrix = filtered_corr_matrix.loc[parameter_order, parameter_order]

    # Set diagonal to 1.00 explicitly
    np.fill_diagonal(filtered_corr_matrix.values, 1)

    print("Generating heatmap...")
    fig = px.imshow(
        filtered_corr_matrix,
        text_auto=".2f",
        color_continuous_scale="RdBu",
        zmin=-1,
        zmax=1,
        labels={"x": labels[0], "y": labels[1], "color": "Correlation Coefficient"},
        title=title,
    )

    fig.update_layout(
        title=dict(font=dict(size=20), x=0.5),
        xaxis=dict(tickangle=45, title=None, tickfont=dict(size=12)),
        yaxis=dict(title=None, tickfont=dict(size=12)),
        autosize=False,
        width=1200,
        height=1000,
        margin=dict(l=100, r=100, t=100, b=100),
    )

    fig.show()
    return filtered_corr_matrix


# Main function
if __name__ == "__main__":
    print("Upload data files for the WWTP unit processes in sequence.")
    process_files, process_labels = upload_files_sequentially()

    for i in range(len(process_files) - 1):
        print(f"Generating heatmap between {process_labels[i]} and {process_labels[i + 1]}...")
        merged_df = align_datasets(process_files[i], process_files[i + 1], process_labels[i], process_labels[i + 1])
        title = f"Interactive Heatmap: {process_labels[i]} vs {process_labels[i + 1]}"
        labels = (process_labels[i], process_labels[i + 1])
        generate_interactive_heatmap(merged_df, title, labels)


Upload data files for the WWTP unit processes in sequence.
Enter the full path to the next process file (e.g., Influent): /content/Influent Parameters.csv
What is the name of this process? (e.g., Influent): Influent
Do you want to add another process file? (yes/no): yes
Enter the full path to the next process file (e.g., Influent): /content/Oxidation Ditch Effluent Parameters.xlsx
What is the name of this process? (e.g., Influent): Oxidation Ditch Effluent
Do you want to add another process file? (yes/no): yes
Enter the full path to the next process file (e.g., Influent): /content/Plant Effluent Parameters.xlsx
What is the name of this process? (e.g., Influent): Effluent
Do you want to add another process file? (yes/no): no
Generating heatmap between Influent and Oxidation Ditch Effluent...
Aligning datasets for Influent and Oxidation Ditch Effluent...
Reading file: /content/Influent Parameters.csv
Applying Z-Score Method to filter outliers...
Finished processing file: /content/Influen

Generating heatmap between Oxidation Ditch Effluent and Effluent...
Aligning datasets for Oxidation Ditch Effluent and Effluent...
Reading file: /content/Oxidation Ditch Effluent Parameters.xlsx
Applying Z-Score Method to filter outliers...
Finished processing file: /content/Oxidation Ditch Effluent Parameters.xlsx
Reading file: /content/Plant Effluent Parameters.xlsx
Applying Z-Score Method to filter outliers...
Finished processing file: /content/Plant Effluent Parameters.xlsx
Datasets aligned for Oxidation Ditch Effluent and Effluent.
Bootstrapping correlation matrices...
Bootstrapping iteration 0/500...
Bootstrapping iteration 100/500...
Bootstrapping iteration 200/500...
Bootstrapping iteration 300/500...
Bootstrapping iteration 400/500...
Bootstrapping iteration 0/500...
Bootstrapping iteration 100/500...
Bootstrapping iteration 200/500...
Bootstrapping iteration 300/500...
Bootstrapping iteration 400/500...
Bootstrapping iteration 0/500...
Bootstrapping iteration 100/500...
Boots